# Linkedin Profile Scrapper
*This is **NOT** a real project, it's only used to learning the scrapping concept.*

### Problem Statement:
A school need to know how many and who are the LinkedIn Alumnis.

### Solution:
A web scrapping to find all users in Alumnis section on LinkedIn's School and save it ins a CSV file.

### Libs
- OS - To manipulate local variables
- Time - To wait page loads
- Pandas - To load and manipulate data
- Selenium - To emulate and navegate in a browser
- Scrapy - To scrap data from a website

## Step 1 - Importing libs and load local variables


In [316]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from scrapy import Selector
import random

login_username = os.getenv('LINKEDIN_USER')
password = os.getenv('LINKEDIN_PASSWORD')


## Step 2 - Access url of School's LinkedIn
2.1 Use local variables do login\
2.2 Access the url\
2.3 Scroll a couple of times (important, the number of scroll depends on number of alumnis)


*to do: Get the number of Alumnis to determine the number of scrolls.*

In [317]:
driver = webdriver.Chrome(r"Users\Felipe Mayer\Downloads\chromedriver_win32")

driver.get("https://linkedin.com/uas/login")
  
# waiting for the page to load
time.sleep(5)
  
# entering username
username = driver.find_element(By.XPATH, "//input[@id='username']")

# Enter Your Email Address
username.send_keys(login_username)  
  
# entering password
pword = driver.find_element(By.XPATH, "//input[@id='password']")
  
# Enter Your Password
pword.send_keys(password)        

# Press login button
driver.find_element(By.XPATH, "//button[@class='btn__primary--large from__button--floating']").submit()

# School Linkedin URL
url = "https://www.linkedin.com/school/cursos-pm3/people/"
  
driver.get(url)

driver.maximize_window()

for x in range(1, 20):
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    try:
        driver.find_element(By.XPATH, './/span[text()="Show more results"]/parent::button').click()
    except:
        pass

src = driver.page_source

# removing from source code the tag <time> to make easier to find profile information
src = src.replace('<time>', '')
src = src.replace('</time>', '')

sel = Selector(text=src)

time.sleep(5)
driver.close()

## Step 3 - Get the Alumnis information
3.1 Transform the page source provided by Driver into a Scrapy Selector.\
3.2 Using xpath, get all alumnis cards (container with information).\
3.3 Iterate over alumni cards and get the main information about them.

In [318]:
urls = sel.xpath('.//section/div/div/div/a[contains(@href, "/in/")]/@href')

df_alumnis = pd.DataFrame()

for i, url in enumerate(urls):
    df_alumnis.loc[i, 'linkedin'] = url.get()

df_alumnis = df_alumnis.drop_duplicates(keep="last").copy()

df_alumnis

,linkedin
0,https://www.linkedin.com/in/gicosilvaalmeida?m...
1,https://www.linkedin.com/in/falacomcaio?miniPr...
2,https://www.linkedin.com/in/guipaulobarros?min...
3,https://www.linkedin.com/in/felipeperazza?mini...
4,https://www.linkedin.com/in/gabrieltarantelli?...
...,...
103,https://www.linkedin.com/in/francielepaulino?m...
104,https://www.linkedin.com/in/kaique-karvelis-pe...
105,https://www.linkedin.com/in/joaonader?miniProf...
106,https://www.linkedin.com/in/marcel-amiach?mini...


In [319]:
for i, row in df_alumnis.head(5).iterrows():
        # Creating a webdriver instance
        driver = webdriver.Chrome(r"Users\Felipe Mayer\Downloads\chromedriver_win32")

        # open browser with current user linkedin url as parameter
        driver.get(row['linkedin'])
        time.sleep(random.randint(2, 5))

        try:
            driver.find_element(By.XPATH, '//*[@id="contextual-sign-in"]/div/section/button').click()
        except:
            pass

        src = driver.page_source

        df_alumnis.loc[i, 'selector'] = Selector(text=src)

        driver.close()
        time.sleep(random.randint(2, 5))

df_alumnis


,linkedin,selector
0,https://www.linkedin.com/in/gicosilvaalmeida?m...,"<Selector xpath=None data='<html lang=""en""><he..."
1,https://www.linkedin.com/in/falacomcaio?miniPr...,"<Selector xpath=None data='<html lang=""en""><he..."
2,https://www.linkedin.com/in/guipaulobarros?min...,"<Selector xpath=None data='<html lang=""en""><he..."
3,https://www.linkedin.com/in/felipeperazza?mini...,"<Selector xpath=None data='<html lang=""en""><he..."
4,https://www.linkedin.com/in/gabrieltarantelli?...,"<Selector xpath=None data='<html lang=""en""><he..."
...,...,...
103,https://www.linkedin.com/in/francielepaulino?m...,NaN
104,https://www.linkedin.com/in/kaique-karvelis-pe...,NaN
105,https://www.linkedin.com/in/joaonader?miniProf...,NaN
106,https://www.linkedin.com/in/marcel-amiach?mini...,NaN


In [320]:
df_alumnis = df_alumnis.loc[~df_alumnis['selector'].isna()].copy()

for i, row in df_alumnis.iterrows():

    if row['selector'] == 'Private profile':
        df_alumnis.loc[i, 'status'] = 'Private'
    else:
        df_alumnis.loc[i, 'status'] = 'Public'
        df_alumnis.loc[i, 'name'] = row['selector'].xpath('//h1/text()').get(default='').replace('\n', ' ').strip()
        df_alumnis.loc[i, 'headline'] = row['selector'].xpath('//h2/text()').get(default='').replace('\n', ' ').strip()
        df_alumnis.loc[i, 'local'] = row['selector'].xpath('//h3/div/text()').get(default='').replace('\n', ' ').strip()
        df_alumnis.loc[i, 'about'] = row['selector'].xpath('//h2[contains(text(),"About")]/following-sibling::div/p/text()').get(default='').replace('\n', ' ').strip()
        df_alumnis.loc[i, 'job_title'] = row['selector'].xpath('//section[contains(@class,"experience")]/div/ul/li[1]/div/h3/text()').get(default='').replace('\n', ' ').strip()
        df_alumnis.loc[i, 'current_company'] = row['selector'].xpath('//section[contains(@class,"experience")]/div/ul/li[1]/div/h4/a/text()').get(default='').replace('\n', ' ').strip()
        df_alumnis.loc[i, 'job_period'] = row['selector'].xpath('//section[contains(@class,"experience")]/div/ul/li[1]/div/div/p/span/text()').get(default='').replace('\n', ' ').strip()
        df_alumnis.loc[i, 'job_duration'] =  row['selector'].xpath('//section[contains(@class,"experience")]/div/ul/li[1]/div/div/p/span/span/text()').get(default='').replace('\n', ' ').strip()
        df_alumnis.loc[i, 'job_description'] = row['selector'].xpath('//section[contains(@class,"experience")]/div/ul/li[1]/div/div/div/div/p/text()').get(default='').replace('\n', ' ').strip()
        df_alumnis.loc[i, 'job_local'] = row['selector'].xpath('//section[contains(@class,"experience")]/div/ul/li[1]/div/div/p[2]/text()').get(default='').replace('\n', ' ').strip()

display(df_alumnis)

,linkedin,selector,status,name,headline,local,about,job_title,current_company,job_period,job_duration,job_description,job_local
0,https://www.linkedin.com/in/gicosilvaalmeida?m...,"<Selector xpath=None data='<html lang=""en""><he...",Public,Giovana Silva Almeida,Report,"São Paulo, São Paulo, Brazil",,,,,,,
1,https://www.linkedin.com/in/falacomcaio?miniPr...,"<Selector xpath=None data='<html lang=""en""><he...",Public,Caio Rosário,Product Manager @ Grupo SBF,"São Paulo, São Paulo, Brazil","Hi! My name is Caio, I am 28 and I currently w...",Product Manager,GRUPO SBF,,2 years 3 months,"As a skilled Product Manager, I have a proven ...","São Paulo, São Paulo, Brazil"
2,https://www.linkedin.com/in/guipaulobarros?min...,"<Selector xpath=None data='<html lang=""en""><he...",Public,Guilherme Barros,Senior Product Manager @ Loggi,"São Paulo, São Paulo, Brazil",,Senior Product Manager,Loggi,,1 year 10 months,,"São Paulo, Brazil"
3,https://www.linkedin.com/in/felipeperazza?mini...,"<Selector xpath=None data='<html lang=""en""><he...",Public,Felipe Perazza,Product Manager | Business Strategy | Marketin...,"São Paulo, São Paulo, Brazil",,Senior Product Manager,Pier Seguradora,,1 year,,"São Paulo, São Paulo, Brazil"
4,https://www.linkedin.com/in/gabrieltarantelli?...,"<Selector xpath=None data='<html lang=""en""><he...",Public,Gabriel Tarantelli,Product Manager na Zup Innovation,"São Paulo, São Paulo, Brazil",,,,,,,
